In [1]:
import sys
sys.path.insert(1, "../python/functions")
sys.path.insert(2, "../python/architecture")
from new_Adam import Adam_test
import reproducible

In [2]:
import tensorflow
class CustomModel(tensorflow.python.keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data
        
        y_pred = tf.Variable(self(x, training=True))  # Forward pass
        with tf.GradientTape(watch_accessed_variables = False) as tape: # watch_acc... macht hier auch keinen Unterschied
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            tape.watch(y_pred)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        gradients = tape.gradient(loss, y_pred)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, y_pred))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [3]:
import numpy as np

# Construct and compile an instance of CustomModel
inputs = tensorflow.python.keras.Input(shape=(32,))
outputs = tensorflow.python.keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer=Adam_test(), loss="mse", metrics=["mae"])

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 19us/sample - loss: 0.5963 - mae: 0.6554
Epoch 2/5
1000/1000 [==============================] - 0s 23us/sample - loss: 0.2495 - mae: 0.4048
Epoch 3/5
1000/1000 [==============================] - 0s 20us/sample - loss: 0.2038 - mae: 0.3633
Epoch 4/5
1000/1000 [==============================] - 0s 28us/sample - loss: 0.1998 - mae: 0.3588
Epoch 5/5
1000/1000 [==============================] - 0s 22us/sample - loss: 0.1961 - mae: 0.3556


Die resultierenden Werte für den Loss und den MAE sind exakt identisch zu der Variante wie sie im Internet steht und wo statt $y_{pred}$ stets $\textit{self.trainable_variables}$ verwendet wird. Auch jeder andere beliebige Wert für $y_{pred}$ ändert absolut nichts, egal ob $y_{pred}$ innerhalb oder außerhalb des $with$-Ausdrucks steht.
#### => Irgendwo anders werden die normalen Parameter weiterhin benutzt.